In [1]:
import os
import glob
import time
import datetime
import sys
import io
import base64
from genericpath import isfile
import hashlib
import shutil
import itertools
import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw
from collections import Counter

import torch
from torch.optim import SGD, Adam, AdamW
from adamp import AdamP
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models
from torchvision import transforms, datasets, models
from torchvision.transforms import Resize, ToTensor, Normalize

import timm
from sklearn.model_selection import StratifiedKFold

In [2]:
from modules.dataset import *
from modules.transformation import *
from modules.imbalancedsampler import *
from modules.loss import *
from modules.config import Config as conf
from modules.config import HyperParameter as params
from modules.inference import *
from modules.utils import *
from modules.train import *
from modules.ensemble import *

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-PCIE-32GB


In [3]:
seed_everything(2021)

이 실험은 seed 2021로 고정되었습니다.


## 1. Prepare dataset

- age의 기준 中 60세 이상을 58세 이상으로 변경
- Stratified K-Fold 적용
   - 동일한 사람이 학습/검증 데이터에 모두 들어가있지않기
   - 분포가 특정 클래스에 편향되어 있지 않기
- transform 적용
    - centercrop(224)
    - normalize

In [4]:
dataset = PreprocessedDataset(conf.train_dir + '/train.csv', False, 'Label')

2700it [00:05, 494.49it/s]


In [5]:
transform = get_transforms()
train_tf, val_tf, test_tf = transform['train'], transform['val'], transform['test']

I'm in simple train
I'm in simple val


- Stratified K-Fold 적용 결과

In [6]:
dataset.train_df['Fold'].value_counts()

4    3780
3    3780
2    3780
1    3780
0    3780
Name: Fold, dtype: int64

## 2. Train

In [7]:
criterion= F1Loss()

In [8]:
start = time.time()

for fold_idx in tqdm.tqdm(range(conf.n_fold)):
    train_dataset = TransformedDataset(dataset, train_tf, fold_idx)
    val_dataset = TransformedDataset(dataset, val_tf, fold_idx)
    
    train_loader = DataLoader(train_dataset, 
                              batch_size=params.BATCH_SIZE, 
                              sampler=ImbalancedDatasetSampler(train_dataset),
                              num_workers=params.NUM_WORKERS)
    
    val_loader = DataLoader(val_dataset, 
                            batch_size=params.BATCH_SIZE, 
                            shuffle=False,
                            num_workers=params.NUM_WORKERS)
    
    print("="*100)
    print("Done preparing training & validation set\nStart Training")
    print("="*100)
    
    model = timm.create_model('resnet50', pretrained=True, num_classes=18)
    if torch.cuda.is_available():
        model.cuda()
        
    optimizer = AdamW(model.parameters(), lr=params.lr, weight_decay=5e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=2, eta_min=0.)
    
    train_and_validate(model, criterion, optimizer, scheduler, train_loader, len(val_dataset), val_loader)

    print("="*100)
    print(f"fold: {fold_idx}, time: " + sec_to_str(time.time()-start))
    print("="*100)
    
    torch.save(model, open(f"resnet50_fold_{fold_idx}", "wb"))
    start = time.time()
    
print("="*100)
print("DONE")
print("="*100)

  0%|          | 0/5 [00:00<?, ?it/s]

Done preparing training & validation set
Start Training
Epoch[0/5](20/237) || training loss 0.946 || training accuracy 12.19% || lr [1e-05]
Epoch[0/5](40/237) || training loss 0.9399 || training accuracy 23.83% || lr [1e-05]
Epoch[0/5](60/237) || training loss 0.9283 || training accuracy 34.45% || lr [1e-05]
Epoch[0/5](80/237) || training loss 0.9019 || training accuracy 42.50% || lr [1e-05]
Epoch[0/5](100/237) || training loss 0.8535 || training accuracy 48.05% || lr [1e-05]
Epoch[0/5](120/237) || training loss 0.8056 || training accuracy 51.02% || lr [1e-05]
Epoch[0/5](140/237) || training loss 0.755 || training accuracy 57.89% || lr [1e-05]
Epoch[0/5](160/237) || training loss 0.7108 || training accuracy 55.08% || lr [1e-05]
Epoch[0/5](180/237) || training loss 0.6679 || training accuracy 59.06% || lr [1e-05]
Epoch[0/5](200/237) || training loss 0.6419 || training accuracy 61.88% || lr [1e-05]
Epoch[0/5](220/237) || training loss 0.5932 || training accuracy 64.92% || lr [1e-05]
Calc

 20%|██        | 1/5 [08:24<33:37, 504.43s/it]

Done preparing training & validation set
Start Training
Epoch[0/5](20/237) || training loss 0.9466 || training accuracy 10.78% || lr [1e-05]
Epoch[0/5](40/237) || training loss 0.9403 || training accuracy 26.41% || lr [1e-05]
Epoch[0/5](60/237) || training loss 0.9266 || training accuracy 44.22% || lr [1e-05]
Epoch[0/5](80/237) || training loss 0.8977 || training accuracy 50.39% || lr [1e-05]
Epoch[0/5](100/237) || training loss 0.8493 || training accuracy 53.83% || lr [1e-05]
Epoch[0/5](120/237) || training loss 0.799 || training accuracy 57.03% || lr [1e-05]
Epoch[0/5](140/237) || training loss 0.747 || training accuracy 61.41% || lr [1e-05]
Epoch[0/5](160/237) || training loss 0.6884 || training accuracy 60.78% || lr [1e-05]
Epoch[0/5](180/237) || training loss 0.6431 || training accuracy 63.83% || lr [1e-05]
Epoch[0/5](200/237) || training loss 0.6036 || training accuracy 65.39% || lr [1e-05]
Epoch[0/5](220/237) || training loss 0.5528 || training accuracy 68.67% || lr [1e-05]
Calc

 40%|████      | 2/5 [16:23<24:50, 496.72s/it]

...

Done preparing training & validation set
Start Training
Epoch[0/5](20/237) || training loss 0.9464 || training accuracy 10.00% || lr [1e-05]
Epoch[0/5](40/237) || training loss 0.9396 || training accuracy 24.69% || lr [1e-05]
Epoch[0/5](60/237) || training loss 0.9294 || training accuracy 38.28% || lr [1e-05]
Epoch[0/5](80/237) || training loss 0.9042 || training accuracy 46.64% || lr [1e-05]
Epoch[0/5](100/237) || training loss 0.8524 || training accuracy 50.47% || lr [1e-05]
Epoch[0/5](120/237) || training loss 0.8028 || training accuracy 51.95% || lr [1e-05]
Epoch[0/5](140/237) || training loss 0.7512 || training accuracy 54.37% || lr [1e-05]
Epoch[0/5](160/237) || training loss 0.7026 || training accuracy 58.98% || lr [1e-05]
Epoch[0/5](180/237) || training loss 0.6679 || training accuracy 58.83% || lr [1e-05]
Epoch[0/5](200/237) || training loss 0.6151 || training accuracy 64.22% || lr [1e-05]
Epoch[0/5](220/237) || training loss 0.5893 || training accuracy 65.16% || lr [1e-05]
Ca

 80%|████████  | 4/5 [31:47<07:59, 479.15s/it]

Done preparing training & validation set
Start Training
Epoch[0/5](20/237) || training loss 0.947 || training accuracy 9.06% || lr [1e-05]
Epoch[0/5](40/237) || training loss 0.9393 || training accuracy 26.41% || lr [1e-05]
Epoch[0/5](60/237) || training loss 0.9267 || training accuracy 37.42% || lr [1e-05]
Epoch[0/5](80/237) || training loss 0.9011 || training accuracy 39.92% || lr [1e-05]
Epoch[0/5](100/237) || training loss 0.8617 || training accuracy 40.00% || lr [1e-05]
Epoch[0/5](120/237) || training loss 0.8069 || training accuracy 44.06% || lr [1e-05]
Epoch[0/5](140/237) || training loss 0.7507 || training accuracy 47.27% || lr [1e-05]
Epoch[0/5](160/237) || training loss 0.713 || training accuracy 49.61% || lr [1e-05]
Epoch[0/5](180/237) || training loss 0.6777 || training accuracy 54.77% || lr [1e-05]
Epoch[0/5](200/237) || training loss 0.6387 || training accuracy 60.23% || lr [1e-05]
Epoch[0/5](220/237) || training loss 0.5941 || training accuracy 61.48% || lr [1e-05]
Calcu

100%|██████████| 5/5 [39:29<00:00, 474.00s/it]

DONE


## 3. Inference with K-Fold Models

In [11]:
modelA = torch.load(open(os.path.join(conf.model_dir, 'resnet50_fold_0'), "rb")).to(device)
modelB = torch.load(open(os.path.join(conf.model_dir, 'resnet50_fold_1'), "rb")).to(device)
modelC = torch.load(open(os.path.join(conf.model_dir, 'resnet50_fold_2'), "rb")).to(device)
modelD = torch.load(open(os.path.join(conf.model_dir, 'resnet50_fold_3'), "rb")).to(device)
modelE = torch.load(open(os.path.join(conf.model_dir, 'resnet50_fold_4'), "rb")).to(device)

model = FoldEnsemble(modelA, modelB, modelC, modelD, modelE)
model = model.to(device)
model.eval()

FoldEnsemble(
  (modelA): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [38]:
inference(model, test_tf, 'resnet50.csv')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-PCIE-32GB
Inference Done
